In [7]:
def predict_number(n):
    while n > 0:
        highest_power_of_2 = 1 << (n.bit_length() - 1)
        n = n - highest_power_of_2
        if n == 0:
            return 0
        elif n == highest_power_of_2 // 2:
            return 1
        elif n == highest_power_of_2 - 1:
            return 2

    return 0

# Reading input and processing
print(predict_number(5))

0
